# **BERT Twitter Sentiment Analysis**


In [1]:
from google.colab import drive # for Google Colab
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.0MB 8.1MB/s 
     |████████████████████████████████| 890kB 50.5MB/s 
     |████████████████████████████████| 3.2MB 50.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=223252823155b7158c7f38e861a77d77c54ab13c4e1be14ae635d43e9917d535
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
import numpy as np
from torchtext.legacy import data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## **0. BERT Tokenizer**

First, import BERT tokenizer.

In [4]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Then, get the indexes of the special tokens.

In [5]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

Now define a tokenize function.

In [6]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

Define the label field as before.

## **1. Dataset Preparation**
The first column contains the sentiments and the last column contains the tweets.

In [7]:
# Read in data into a dataframe
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tweet-dataset.csv", engine="python", header=None)

df.head(5)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
# Count the number of tweets per sentiment
df[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

The dataset consists of two sentiments (0 - negative, 4 - positive)

In [9]:
# Model the sentiments as binary (0 - negative, 1 - positive)
df[0]=df[0].replace(to_replace=4,value=1)
df[0].value_counts()

1    800000
0    800000
Name: 0, dtype: int64

In [10]:
# Save a subset as a smaller dataset from training
df.sample(100000).to_csv("sentiment140-small.csv", header=None, index=None)

## **2. Preprocessing**

In [11]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [12]:
import torchtext

max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

# Map data to fields
fields = [('label', LABEL), ('id',None),('date',None),('query',None),
      ('name',None), ('text', TEXT),('category',None)]

# Apply field definition to create torch dataset
dataset = torchtext.legacy.data.TabularDataset(
        path="sentiment140-small.csv",
        format="CSV",
        fields=fields,
        skip_header=False)

# Split data into train, test, validation sets
(train_data, test_data, valid_data) = dataset.split(split_ratio=[0.8,0.1,0.1])

print("Number of train data: {}".format(len(train_data)))
print("Number of test data: {}".format(len(test_data)))
print("Number of validation data: {}".format(len(valid_data)))

Number of train data: 80000
Number of test data: 10000
Number of validation data: 10000


In [13]:
# An example from the training set
print(vars(train_data.examples[0]))

{'label': '0', 'text': [1030, 2028, 8405, 2620, 2549, 8038, 1045, 2018, 1996, 3602, 2035, 2772, 1998, 2074, 2245, 1045, 2876, 1005, 1056, 2342, 2000, 2404, 2009, 2041, 2127, 2101, 1999, 2154]}


### **Build Vocabulary**
Build the vocabulary for the training set using pre-trained GloVe embeddings.
GloVe embeddings were trained on 6 billion tokens and the embeddings are 100-dimensional.

In [14]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1})


In [15]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)


bert = BertModel.from_pretrained('bert-base-uncased')

## **3. Architecture**

In [16]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

### **Create Model**

In [17]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [18]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

<a id='section4'></a>
# **4. Training**

In [19]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

# binary cross entropy
criterion = nn.BCEWithLogitsLoss()

# Use GPU
model = model.to(device)
criterion = criterion.to(device)

In [20]:
# Helper functions

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### **LSTM Training**

In [22]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model-bert.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 5m 3s
	Train Loss: 0.481 | Train Acc: 76.65%
	 Val. Loss: 0.415 |  Val. Acc: 80.78%
Epoch: 02 | Epoch Time: 5m 7s
	Train Loss: 0.415 | Train Acc: 81.13%
	 Val. Loss: 0.402 |  Val. Acc: 82.18%
Epoch: 03 | Epoch Time: 5m 8s
	Train Loss: 0.392 | Train Acc: 82.35%
	 Val. Loss: 0.395 |  Val. Acc: 82.11%
Epoch: 04 | Epoch Time: 5m 7s
	Train Loss: 0.371 | Train Acc: 83.57%
	 Val. Loss: 0.395 |  Val. Acc: 82.77%
Epoch: 05 | Epoch Time: 5m 8s
	Train Loss: 0.350 | Train Acc: 84.59%
	 Val. Loss: 0.405 |  Val. Acc: 83.01%


## **5. Test**

In [23]:
model.load_state_dict(torch.load('model-bert.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.386 | Test Acc: 82.79%
